In [1]:
import os
os.environ["PYTHON_EXEC"] = "/Users/jessieyu/.pyenv/versions/qka-demo/bin/python3"
os.environ["NTC_DOC_FILE"] = "runtime/qka_doc.json"
os.environ["NTC_PROGRAM_FILE"] = "runtime/qka_runtime_outer.py"

# List available quantum programs

In [2]:
from qiskit import IBMQ

provider = IBMQ.load_account()
provider.runtime.programs()

Runtime Program Quantum-Kernel-Alignment:
  Description: Quantum kernel alignment algorithm that learns, on a given dataset, a quantum kernel maximizing the SVM classification margin.
  Parameters:
    feature_map:
      description: An instance of FeatureMapForrelation in dictionary format used to map classical data into a quantum state space.
      type: dict
    data:
      description: NxD array of training data, where N is the number of samples and D is the feature dimension.
      type: numpy.ndarray
    labels:
      description: Nx1 array of +/-1 labels of the N training samples.
      type: numpy.ndarray
    lambda_initial:
      description: Initial parameters of the quantum feature map.
      type: numpy.ndarray
    spsa_steps:
      description: Number of SPSA optimization steps.
      type: int
    C:
      description: Penalty parameter for the soft-margin support vector machine.
      type: float
  Returns:
    aligned_kernel_parameters:
      description: The optimized 

# Invoke Quantum Kernel Alignment program

In [3]:
import numpy as np
from qka.featuremaps import FeatureMapForrelation

num_samples=5  # number of samples per class in the input data
num_features=2 # number of features in the input data
depth=2        # depth of the feature map circuit
C=1            # SVM soft-margin penalty
spsa_steps=2   # number of SPSA iterations

# Define the feature map and its initial parameters:
fm = FeatureMapForrelation(feature_dimension=num_features, depth=depth, entangler_map=None)
lambda_initial = np.random.uniform(-1,1, size=(fm._num_parameters))

# create random test data and labels:
x_train = np.random.rand(2*num_samples, num_features)
y_train = np.concatenate((-1*np.ones(num_samples), np.ones(num_samples)))

backend = provider.backend.ibmq_qasm_simulator

In [4]:
def interim_result_callback(interim_result):
    print(f"interim result: {interim_result}\n")

In [5]:
# Call QKA runtime program.
runtime_params = {
    'feature_map': fm.to_dict(),
    'data': x_train,
    'labels': y_train,
    'lambda_initial': lambda_initial,
    'spsa_steps': 2,
    'C': C
}
options = {'backend_name': backend.name()}
result = provider.runtime.run(program_name="Quantum-Kernel-Alignment",
                              options=options,
                              params=runtime_params,
                              callback=interim_result_callback,
                             ).result()
print("final result:")
print(f"aligned_kernel_parameters: {result['aligned_kernel_parameters']}")
print(f"aligned_kernel_matrix: {result['aligned_kernel_matrix']}")

interim result: {'cost': 8.1525789395065, 'lambda': array([ 0.60879331,  0.0966291 ,  0.01985885, -0.14710885]), 'cost_plus': 7.995116904546321, 'cost_minus': 8.310040974466677, 'cost_final': 8.1525789395065}

interim result: {'cost': 7.6847166025258975, 'lambda': array([ 0.69607475,  0.18391054, -0.06742259, -0.23439029]), 'cost_plus': 6.449514745412854, 'cost_minus': 8.919918459638941, 'cost_final': 7.6847166025258975}

final result:
aligned_kernel_parameters: [ 0.13048681  0.02805396 -0.00475637 -0.03814991]
aligned_kernel_matrix: [[1.         0.36914062 0.31054688 0.93847656 0.47167969 0.87890625
  0.65234375 0.25585938 0.75585938 0.4453125 ]
 [0.36914062 1.         0.09179688 0.265625   0.24804688 0.57324219
  0.68652344 0.41894531 0.63476562 0.29296875]
 [0.31054688 0.09179688 1.         0.32617188 0.2109375  0.23535156
  0.015625   0.52441406 0.04492188 0.20117188]
 [0.93847656 0.265625   0.32617188 1.         0.36035156 0.75976562
  0.578125   0.16699219 0.69042969 0.32128906]
